In [1]:
from os import listdir
from os.path import join
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


import fiona
import rasterio
import rasterio.mask
import os

In [16]:
infolder = 'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\009_mosaicing_done\\'

outfolder = 'N:\\Projekt_PROSPER_RO\\Data_RS\\Sentinel2\\009d_int\\'




In [17]:
files = []
for file in os.listdir(infolder):
    if file.endswith(".tif"):
        files.append(file)

print('number of images: {} \n first image name: {}'.format(len(files), files[0]))

number of images: 19 
 first image name: 20180420_S2B_MSIL2A_mosaic.tif


In [26]:
dataset = rasterio.open(infolder+files[2])


In [27]:
dataset.meta

{'driver': 'GTiff',
 'dtype': 'float32',
 'nodata': None,
 'width': 9122,
 'height': 7225,
 'count': 13,
 'crs': CRS.from_dict(init='epsg:32633'),
 'transform': Affine(9.999537072399168, 0.0, 265083.52981687954,
        0.0, -9.999537072399168, 6029851.70325219)}

In [28]:
metas = dataset.meta
metas.update({"dtype": 'uint16'})
metas

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': None,
 'width': 9122,
 'height': 7225,
 'count': 13,
 'crs': CRS.from_dict(init='epsg:32633'),
 'transform': Affine(9.999537072399168, 0.0, 265083.52981687954,
        0.0, -9.999537072399168, 6029851.70325219)}

In [32]:
with rasterio.open(outfolder+'test2.tif', "w", **metas) as dest:
    dest.write(dataset)

ValueError: Source shape (1,) is inconsistent with given indexes 1

In [29]:
def ndvi_index(infolder,outfolder,image, band1, band2):
    dataset = rasterio.open(infolder+image)
    
    b1 = dataset.read(band1)
    b2 = dataset.read(band2)
    a = b2-b1
    b = b2+b1
       
    ndvi = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    
    with rasterio.open(outfolder+image[:-4]+'_ndvi.tif',
                   'w',
                   driver='GTiff',
                   height=ndvi.shape[0],
                   width=ndvi.shape[1],
                   count=1,
                   dtype=ndvi.dtype,
                   crs=dataset.crs,
                   transform=dataset.transform
                  ) as dst:
        dst.write(ndvi,1)

In [30]:
for image in files:
    ndvi_index(infolder,outfolder,image,band_red,band_nir)

In [31]:
files = []
for file in os.listdir(outfolder):
    if file.endswith(".tif"):
        files.append(file)

print('number of images: {} \n first image name: {}'.format(len(files), files[0]))

number of images: 19 
 first image name: 20180420_S2B_MSIL2A_mosaic_ndvi.tif


In [32]:
def crop_image(shape, image, output):
    
    with fiona.open(shape, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    
    with rasterio.open(image) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta
    
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
    
    with rasterio.open(output, "w", **out_meta) as dest:
        dest.write(out_image)

In [33]:
for i in files:
    crop_image(inshape, outfolder+i, outfolder+i[:-4]+'_subset.tif')